In [1]:
import os
from time import perf_counter
import pandas as pd
from src/config import FOLDER_DATA

result_folders = ['mq_out', 'Hela']

In [2]:
peptides = {}
t0 = perf_counter()
for _folder in result_folders:
    _file = os.path.join(FOLDER_DATA, _folder, 'peptide_intensities')
    try: 
        peptides[_folder] = pd.read_pickle(_file + '.pkl')
    except FileNotFoundError:
        peptides[_folder] = pd.read_csv(_file)
print(f"File loading took {perf_counter() - t0:.1f} seconds")
peptides.keys()    

File loading took 17.1 seconds


dict_keys(['mq_out', 'Hela'])

In [3]:
MIN_N_PEPTIDES = 25000
for i, (key, data_set) in enumerate(peptides.items()):
    _df = data_set.notna().sum(axis=1).to_frame(name=key)
    mask = _df[key] >= MIN_N_PEPTIDES
    _df = _df.loc[mask].describe()
    if i == 0:
        df = _df
    else:
        df = df.join(_df)
df

,mq_out,Hela
count,291.000000,381.000000
mean,43904.529210,40865.601050
std,4990.084257,5480.106054
min,25393.000000,25122.000000
25%,40976.500000,37623.000000
50%,45237.000000,41808.000000
75%,47260.500000,44467.000000
max,54316.000000,53296.000000


In [4]:
with pd.ExcelWriter('tables/maxquant_file_analysis.xlsx') as writer:
    df.to_excel(writer, sheet_name=f'min_{MIN_N_PEPTIDES}_pep', float_format="%.2f")
df.to_excel(f'tables/maxquant_file_analysis_min_{MIN_N_PEPTIDES}_pep.xlsx', float_format="%.2f")

In [5]:
for i, (key, data_set) in enumerate(peptides.items()):
    print(f"Peptide intesities in {key:6} take up {data_set.memory_usage(deep=False).sum() / 1000000:7.2f} MB of memory")

Peptide intesities in mq_out take up  377.85 MB of memory
Peptide intesities in Hela   take up  750.95 MB of memory


>Note: This should correspond to the size of the pickled DataFrame on disk.